# Week 7 Session 2: Tracing Function Calling & Streamlit Apps

## Learning Objectives
By the end of this session, you will:
1. Trace function calling with Langfuse for full visibility
2. Build a traced Streamlit chatbot with tools
3. Debug AI behavior by examining traces
4. Understand how to monitor production applications
5. Use traces to improve AI reliability

## Prerequisites
- Completed Week 7 Session 1 (Langfuse basics)
- Week 5 knowledge (function calling)
- Week 6 knowledge (Streamlit)

## Why This Matters

**Session 1**: You learned to trace simple functions

**Session 2**: Now trace REAL applications!

### Real-World Debugging Scenarios

**Problem 1**: "My AI chatbot called the wrong tool!"
- ❌ Without tracing: Guess why it happened
- ✅ With tracing: See exactly what the AI received and why it chose that tool

**Problem 2**: "Function calling works in notebook but fails in Streamlit!"
- ❌ Without tracing: Add print statements everywhere
- ✅ With tracing: Compare traces from both environments

**Problem 3**: "AI sometimes gives wrong calculations!"
- ❌ Without tracing: Cannot reproduce the issue
- ✅ With tracing: See the exact tool calls and parameters

**This session teaches you to build observable AI apps!** 🚀

---

## Setup

Make sure you completed Session 1 setup!

In [ ]:
# Standard imports
import os
from dotenv import load_dotenv
from google import genai
from langfuse import observe, get_client

# Load environment
load_dotenv()

# Initialize Gemini
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
MODEL = "gemini-2.5-flash-lite"

print("✅ All imports ready!")
print("✅ Langfuse configured!")
print("🔍 All function calls will be traced automatically")

---

## Part 1: Tracing Function Calling

### The Challenge

When AI uses tools, you need to see:
1. **Which tool** the AI chose to call
2. **What parameters** it extracted from user input
3. **What result** the tool returned
4. **How the AI** used that result in its response

Without tracing, this is a **complete black box**! 📦❌

### Creating Traced Tools

Let's create some simple tools and trace them.

In [ ]:
@observe()  # Trace this tool!
def calculate_tip(bill_amount: float, tip_percentage: float) -> float:
    """Calculate tip amount based on bill and percentage.
    
    Args:
        bill_amount: Total bill before tip in dollars
        tip_percentage: Tip percentage (e.g., 15 for 15%)
    
    Returns:
        Tip amount in dollars
    """
    result = bill_amount * (tip_percentage / 100)
    print(f"💰 Calculating tip: ${bill_amount} × {tip_percentage}% = ${result:.2f}")
    return result

@observe()  # Trace this too!
def split_bill(total_amount: float, num_people: int) -> float:
    """Split a bill evenly among people.
    
    Args:
        total_amount: Total amount to split in dollars
        num_people: Number of people to split among
    
    Returns:
        Amount per person in dollars
    """
    if num_people <= 0:
        return 0.0
    result = total_amount / num_people
    print(f"🧮 Splitting ${total_amount} among {num_people} people = ${result:.2f} each")
    return result

@observe()  # And this!
def multiply(a: float, b: float) -> float:
    """Multiply two numbers.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        Product of a and b
    """
    result = a * b
    print(f"✖️  Multiplying {a} × {b} = {result}")
    return result

print("✅ Tools defined with tracing!")

### Using Tools with AI (Traced)

Now let's create a traced function that uses these tools with Gemini.

In [ ]:
@observe()  # Trace the AI interaction
def ask_with_tools(prompt: str) -> str:
    """Ask Gemini with access to tools."""
    
    # Create chat with tools
    chat = client.chats.create(
        model=MODEL,
        config={
            'tools': [calculate_tip, split_bill, multiply],
            'system_instruction': 'You are a helpful assistant with access to calculation tools. Use them when appropriate.'
        }
    )
    
    # Send message
    response = chat.send_message(prompt)
    return response.text

# Test it
print("🔄 Testing traced function calling...\n")
result = ask_with_tools("The bill is $85 and I want to leave a 20% tip. How much is the tip?")
print(f"\n📤 AI Response: {result}")
print("\n✅ Check Langfuse dashboard to see the FULL trace!")

### 🎯 Explore the Trace

**Go to Langfuse dashboard now!**

You'll see a tree like this:

```
ask_with_tools (parent)
└── calculate_tip
    └── Input: {bill_amount: 85.0, tip_percentage: 20.0}
    └── Output: 17.0
```

**What you can see:**
1. ✅ The user's original prompt
2. ✅ Which tool the AI decided to call (`calculate_tip`)
3. ✅ The exact parameters it extracted (`bill_amount=85.0, tip_percentage=20.0`)
4. ✅ The tool's return value (`17.0`)
5. ✅ The AI's final response using that value

**This is debugging gold!** ✨

---

## Part 2: Complex Multi-Tool Scenarios

Let's try something harder - a query that needs MULTIPLE tool calls!

In [ ]:
print("🔄 Testing multi-tool scenario...\n")

result = ask_with_tools(
    "The bill is $120, I want an 18% tip, and we're 3 people. How much does each person pay?"
)

print(f"\n📤 AI Response: {result}")
print("\n✅ Check Langfuse - you'll see MULTIPLE tool calls!")

### 🎯 What You'll See in Langfuse

```
ask_with_tools
├── calculate_tip
│   └── Input: {bill_amount: 120.0, tip_percentage: 18.0}
│   └── Output: 21.6
└── split_bill
    └── Input: {total_amount: 141.6, num_people: 3}
    └── Output: 47.2
```

**Notice:**
- The AI called TWO tools
- It calculated tip first: $120 × 18% = $21.60
- Then split total: ($120 + $21.60) ÷ 3 = $47.20
- All automatically traced!

### Why This Is Powerful for Debugging

**Scenario**: User reports "The split bill calculation is wrong!"

**Without tracing**: 
- ❌ Guess which values the AI used
- ❌ Can't see if AI made wrong assumptions
- ❌ Hard to reproduce the issue

**With tracing**:
- ✅ See the EXACT parameters AI extracted
- ✅ Verify each tool's output
- ✅ Identify if problem is in tool logic or AI interpretation

---

## Part 3: Tracing Multi-Turn Conversations with Tools

Now let's combine everything: multi-turn chat + function calling + full tracing!

In [ ]:
@observe()  # Trace each individual turn
def send_message_with_tools(chat, message: str, turn_number: int) -> str:
    """Send a single message with tool access - traced!"""
    print(f"\n👤 User [{turn_number}]: {message}")
    response = chat.send_message(message)
    print(f"🤖 AI [{turn_number}]: {response.text}")
    return response.text

@observe()  # Trace the entire conversation
def have_tool_conversation(messages: list[str], session_id: str = None) -> list[str]:
    """Multi-turn conversation with tool access."""
    
    # Add session tracking
    if session_id:
        langfuse = get_client()
        langfuse.update_current_trace(
            session_id=session_id,
            tags=["conversation", "tool-calling", "multi-turn"]
        )
    
    # Create chat with tools
    chat = client.chats.create(
        model=MODEL,
        config={
            'tools': [calculate_tip, split_bill, multiply],
            'system_instruction': 'You are a helpful restaurant assistant with calculation tools. Always provide an answer to the user'
        }
    )
    
    # Each turn gets its own trace
    responses = []
    for i, message in enumerate(messages, 1):
        response = send_message_with_tools(chat, message, i)
        responses.append(response)
    
    return responses

# Test it
print("🔄 Starting multi-turn conversation with tools...")

conversation = [
    "Hi! Our bill is $150",
    "Can you calculate a 20% tip?",
    "Great! Now split the total among 4 people"
]

responses = have_tool_conversation(conversation, session_id="restaurant_123")
print("\n✅ Conversation traced! Check Langfuse dashboard")

### 🎯 The Full Trace Structure

**In Langfuse, you'll see:**

```
have_tool_conversation (session: restaurant_123)
├── send_message_with_tools (turn 1)
│   └── No tools called (just greeting)
├── send_message_with_tools (turn 2)
│   └── calculate_tip
│       └── Input: {bill_amount: 150.0, tip_percentage: 20.0}
│       └── Output: 30.0
└── send_message_with_tools (turn 3)
    └── split_bill
        └── Input: {total_amount: 180.0, num_people: 4}
        └── Output: 45.0
```

### Why This Structure Matters

**For debugging:**
- 🔍 See which turn used which tools
- 🔍 Track conversation context across turns
- 🔍 Verify AI remembered information from earlier turns

**For monitoring:**
- 📊 Track how many tools are called per conversation
- 📊 Identify conversations with errors
- 📊 Measure performance per turn

---

## Part 4: Building a Traced Streamlit App

Now let's take everything we learned and build a **production-ready** traced chatbot!

### The App Structure

We'll create a Streamlit app that:
1. ✅ Has a chat interface
2. ✅ Uses multiple tools
3. ✅ Traces EVERY interaction
4. ✅ Shows trace info in the UI

### Why Streamlit + Tracing?

**Problem**: Streamlit apps are hard to debug
- State management is complex
- Errors happen in production
- Users can't explain what went wrong

**Solution**: Trace everything!
- See exactly what user typed
- See exactly what AI did
- Reproduce issues easily

### The Code

I've created a complete Streamlit app for you in:
```
Week_7/session_2_apps/traced_chat_app.py
```

**To run it:**
```bash
uv run streamlit run Week_7/session_2_apps/traced_chat_app.py
```

### What Makes It Special?

Every single interaction is traced:
- User messages
- AI responses
- Tool calls
- Errors

**Try these in the app:**
1. "Calculate a 20% tip on $85"
2. "What's 47 times 23?"
3. "The bill is $120, I want 18% tip split among 3 people"

Then check Langfuse to see **everything** that happened!

---

## Part 5: Debugging Real Issues with Traces

Let's see how tracing helps debug actual problems.

### Scenario 1: Wrong Tool Called

**User complaint**: "I asked for a tip calculation but got multiplication!"

In [ ]:
# Deliberately ambiguous prompt
print("🔄 Testing ambiguous prompt...\n")

result = ask_with_tools("What's 15 on 100?")
print(f"\n📤 AI Response: {result}")
print("\n🔍 Check Langfuse: Which tool did it call?")
print("💡 The trace shows you EXACTLY what the AI understood!")

### Debug Process

1. **Check the trace** in Langfuse
2. **See which tool** was called
3. **See what parameters** were extracted
4. **Understand** why AI made that choice
5. **Fix** by improving prompt or tool descriptions

### Scenario 2: Tool Returns Error

What happens when a tool fails?

In [ ]:
# This should cause an issue
print("🔄 Testing edge case...\n")

result = ask_with_tools("Split $100 among 0 people")
print(f"\n📤 AI Response: {result}")
print("\n🔍 Check Langfuse: Did the tool return 0? How did AI handle it?")

### What the Trace Shows

In Langfuse:
- ✅ See the tool was called with `num_people=0`
- ✅ See it returned `0.0`
- ✅ See how AI explained this to the user

**Without tracing**: You'd never know the tool was actually called!

---

## Part 6: Production Monitoring Patterns

### Pattern 1: User Identification

Always tag traces with user info:

In [ ]:
@observe()
def production_ask(prompt: str, user_id: str, environment: str = "production") -> str:
    """Production-ready function with full metadata."""
    
    # Add comprehensive metadata
    langfuse = get_client()
    langfuse.update_current_trace(
        user_id=user_id,
        tags=[environment, "function-calling"],
        metadata={
            "model": MODEL,
            "tools_available": ["calculate_tip", "split_bill", "multiply"],
            "version": "2.0"
        }
    )
    
    # Create chat with tools
    chat = client.chats.create(
        model=MODEL,
        config={
            'tools': [calculate_tip, split_bill, multiply],
            'system_instruction': 'You are a helpful assistant with calculation tools.'
        }
    )
    
    response = chat.send_message(prompt)
    return response.text

# Test with metadata
result = production_ask(
    "Calculate 15% tip on $200",
    user_id="customer_456",
    environment="testing"
)

print(f"Result: {result}")
print("\n✅ Check Langfuse - filter by user_id to see ALL their traces!")

### Pattern 2: Error Tracking

In [ ]:
@observe()
def safe_ask(prompt: str) -> dict:
    """Ask with comprehensive error handling."""
    
    try:
        chat = client.chats.create(
            model=MODEL,
            config={
                'tools': [calculate_tip, split_bill, multiply],
                'system_instruction': 'You are a helpful assistant.'
            }
        )
        
        response = chat.send_message(prompt)
        
        return {
            "success": True,
            "response": response.text,
            "error": None
        }
        
    except Exception as e:
        # Error is automatically logged in trace!
        return {
            "success": False,
            "response": None,
            "error": str(e)
        }

# Test
result = safe_ask("Calculate tip on $50 at 15%")
print(f"Success: {result['success']}")
print(f"Response: {result['response']}")
print("\n✅ Even errors are traced!")

### Pattern 3: A/B Testing

Use tags to compare different approaches:

In [ ]:
@observe()
def ask_variant_a(prompt: str) -> str:
    """Variant A: Shorter system instruction."""
    langfuse = get_client()
    langfuse.update_current_trace(tags=["experiment", "variant-a", "short-instruction"])
    
    chat = client.chats.create(
        model=MODEL,
        config={
            'tools': [calculate_tip, split_bill, multiply],
            'system_instruction': 'You are helpful.'
        }
    )
    return chat.send_message(prompt).text

@observe()
def ask_variant_b(prompt: str) -> str:
    """Variant B: Detailed system instruction."""
    langfuse = get_client()
    langfuse.update_current_trace(tags=["experiment", "variant-b", "detailed-instruction"])
    
    chat = client.chats.create(
        model=MODEL,
        config={
            'tools': [calculate_tip, split_bill, multiply],
            'system_instruction': 'You are a helpful restaurant assistant. Use tools when appropriate to provide accurate calculations.'
        }
    )
    return chat.send_message(prompt).text

# Test both
test_prompt = "I have a $100 bill and want 20% tip for 2 people"

print("Testing variant A...")
result_a = ask_variant_a(test_prompt)
print(f"A: {result_a}\n")

print("Testing variant B...")
result_b = ask_variant_b(test_prompt)
print(f"B: {result_b}\n")

print("✅ Compare in Langfuse: Filter by 'variant-a' vs 'variant-b' tags!")

---

## 🎯 Practice Exercises

### Exercise 1: Add More Tools

**Goal**: Add 2 new tools and trace them.

**TODO**:
1. Create a `subtract(a, b)` tool
2. Create a `calculate_discount(price, discount_percent)` tool
3. Add `@observe()` to both
4. Test them with the AI
5. Check traces in Langfuse

**Success criteria**: See both tools traced in Langfuse

In [ ]:
# Your code here


### Exercise 2: Build a Shopping Assistant

**Goal**: Create a traced shopping assistant with tools.

**TODO**:
1. Create tools: `calculate_discount`, `calculate_tax`, `calculate_shipping`
2. Build a `shopping_assistant` function with these tools
3. Add proper metadata (user_id, session_id)
4. Test with: "I'm buying a $250 item with 10% discount, 8% tax, and $15 shipping"

**Success criteria**: See all 3 tools called in one trace

In [ ]:
# Your code here


### Exercise 3: Debug a Problem

**Goal**: Use traces to find and fix an issue.

**TODO**:
1. Create a tool that sometimes fails (e.g., validates input)
2. Run it with both valid and invalid inputs
3. Use Langfuse to see what happened
4. Improve error handling based on what you see

**Success criteria**: Can explain the failure using traces

In [ ]:
# Your code here


---

## Key Takeaways

### ✅ What You Learned

1. **Function calling is traceable**
   - Every tool call is logged
   - See parameters and return values
   - Understand AI's decision-making

2. **Multi-turn conversations need structure**
   - Each turn = separate trace
   - Parent trace groups the session
   - Track context across turns

3. **Streamlit apps benefit from tracing**
   - Debug production issues
   - See what users actually did
   - Reproduce problems easily

4. **Metadata is critical**
   - Tag with user_id
   - Tag with environment
   - Add version info
   - Enable filtering and analysis

5. **Errors are traced too**
   - Failed tool calls are visible
   - Exceptions are logged
   - Debug faster

### 🎯 Production Checklist

When deploying traced apps:

- ✅ Add `@observe()` to all AI functions
- ✅ Add `@observe()` to all tool functions
- ✅ Include user_id in traces
- ✅ Tag with environment (dev/staging/prod)
- ✅ Add version info in metadata
- ✅ Set up error handling
- ✅ Test trace visibility in dashboard
- ✅ Set up alerts for errors (Langfuse feature)

### 📚 Next Steps

Now you can:
- Build traced applications confidently
- Debug AI behavior systematically
- Monitor production apps
- Compare different approaches
- Improve AI reliability

### 🎯 Resources

- **Langfuse Dashboard**: [https://cloud.langfuse.com](https://cloud.langfuse.com)
- **Documentation**: [https://langfuse.com/docs](https://langfuse.com/docs)
- **Function Calling**: [https://langfuse.com/docs/sdk/python/decorators](https://langfuse.com/docs/sdk/python/decorators)

**You can now build production-ready AI applications! 🚀**